# Assignemt 2: 3D compressor stage design

### Design the NASA rotor 67 compressor stage using the "free vortex design"

The NASA rotor 67 is a well-known validation test case for turbomachinery CFD codes. See for example: 
 * https://ntrs.nasa.gov/api/citations/20050196726/downloads/20050196726.pdf 
 * https://how4.cenaero.be/content/c2-nasa-rotor-67
 
The following data is given:
    
 * Mass flow rate: $\dot m = 35$ 
 * Rotational speed: rpm = 16000
 * Rotor blade tip speed: $U_{tip} = 430$ m/s
 * **Total to total** pressure ratio: 1.6
 * Hub to tip ratio: $r_{hub}/r_{tip} = 0.375$
 * Static inlet conditions: 
     * Pressure $P = 0.8$ bar
     * Temperature $T = 3$ °C

For the working fluid, use the assumption of a calorically perfect gas (constant specific heats) for air with $R_{gas} = 287$ J/kg/K and $\gamma=1.4$. 

Furthermore, for the calculations below, assume:
 * a purely axial inflow velocity at the stage
 * a constant axial velocity across the whole stage
 * a purely axial velocity at the outlet of the stage
 * no losses. 

### Tasks: 

1) design the compressor stage at **mid-span** and calculate: 
    - the inflow area
    - mid span radius
    - axial velocity
    - velocity angles
    - flow coefficient
    - loading factor
    - degree of reaction, etc. 
    Note, the total to total pressure ratio is given, and the Euler equation relates the total enthaly change with the velocity triangles. 

2) Second, perform a 3D design of the compressor stage assuming a **free vortex design**. 
    - Show how the design parameters change along the span (radius of the stage). 
    - Calculate the relative Mach number ($M=w/a$) at the inlet of the stage, $a$ is the speed of sound. 

3) Bonus: compare your design with the original NASA rotor 67 rotor. For example, use the first publication given above as a reference (compare the relative inlet Mach number or other quantities). 

Deliver a report of approximately 4 pages. Group of 2 students. **Do not share your results with the other groups.**


In [ ]:
# Imports
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, interactive_output, Label
import numpy as np
from matplotlib import pyplot as plt

Declaring variables

In [ ]:
mass_flow  =  35 # kg/s
rpm  =  16000 # min^-1
U_tip = 430 # m/s
r_h_r_t = 0.375 # [-]
#rho = 1.204 # kg/m^3
kappa = 1.4 # [-]
R_gas = 287 # J/(kg·K)
T = 273.14+3 # K
p0 = 80000 #Pa
Reaction_midspan = 0.3 #stand in value for now
Phi_midspan = 0.5 # [-]
PR = 1.6 # [-]


Implicit parameters:

In [ ]:
omega = rpm*2*np.pi/60  # rad/s
r_tip = U_tip/omega  # m
r_hub = r_h_r_t*r_tip  # m
U = lambda r: omega * r  # m/s
rho = p0/R_gas/T

inflowArea = np.pi * (np.power( r_tip,2 ) - np.power( r_hub,2 ))  # m^2
print(f'Inflow area: {inflowArea:.2f} m^2')

c_ax = mass_flow / (inflowArea * rho) # m/s
print(f'Axial velocity: {c_ax:.2f} m/s')
machAbsolute = c_ax/np.sqrt(kappa*R_gas*T)  # [-]

r_midspan = (r_tip**2 + r_hub**2)**(1/2)  # m
print(f'Midspan radius: {r_midspan:.2f} m')

Since we have been given the value of pressure ratio we can use it to find $\Delta h_{01}$ and thus the loading factor can be calculated. The free vortex parameter $A$ can be found from Euler equations:
$$
\Delta h=U_{r=mid}c_{\theta 2}
$$
since $c_{\theta 1}=0$
We can arrive to an expression for $c_{\theta 2}$ at midspan and define the parameter $A$


Additionally, by definition
$$\psi=\frac{\Delta h_{0}}{U^{2}}$$
And
$$\phi=\frac{c_{x}}{U}$$

In [ ]:
cp = kappa * R_gas / (kappa - 1.0)
T0_in = T + 0.5 * c_ax**2 / cp  # K
T0_out = T0_in * PR ** ((kappa - 1.0) / kappa)
delta_h0 = cp * (T0_out - T0_in)

psi = lambda r: delta_h0 / (U(r)**2)
print(f'Loading factor: {psi(r_midspan):.3f}')
phi = lambda r: c_ax / U(r)
print(f'Flow coefficient: {phi(r_midspan):.3f}')

c_theta2_mid = delta_h0 / U(r_midspan)
A2 = r_midspan * c_theta2_mid
print(f'free vortex parameter {A2:.3f} m^2/s')

Blade angles calculation

In [ ]:
def calculateBladeAngles(r:float) -> tuple:

    c_theta_2 = A2 / r

    beta_1 = np.arctan( U(r) / c_ax )
    
    beta_2 = np.arctan((-c_theta_2 + U(r)) / c_ax)
    alfa_2  =  np.arctan((U(r) - c_ax * np.tan(beta_2)) / c_ax)

    return beta_1, alfa_2, beta_2

The degree of reaction is then 
$$
R={\frac{1}{2}}-{\frac{\phi}{2}}(\tan\alpha_{1}+\tan\beta_{2})
$$
Since we assume $\alpha \equiv 0$ 
$$
R={\frac{1}{2}}-{\frac{\phi}{2}}\tan\beta_{2}
$$


In [ ]:
degreeOfReaction = lambda r: 1/2 - phi(r) * np.tan(calculateBladeAngles(r)[2]) / 2
print(f'Degree of reaction at midspan: {degreeOfReaction(r_midspan):.3f}')

The code below was introduced at the lecture and can be used for the assignment.

In [ ]:
def compressor_blades(r:float) -> tuple :
    
    UPlot = U(r)/100

    A = np.array([[1, 1], [1, -1]]) # Coefficient matrix
    b = np.array([(1-2*degreeOfReaction(r_midspan))/phi(r_midspan), (psi(r_midspan)+1)/phi(r_midspan)]) # Right-hand side vector
    x = np.linalg.solve(A, b) # Solve for unknowns
    
    # Bezier 2nd order for stator and rotor blades
    bezier_parameter = np.linspace(0, 1, 21)
    BezierMatrix = np.column_stack(((1-bezier_parameter)**2, 2*bezier_parameter*(1-bezier_parameter),  bezier_parameter**2))
    
    # Profile thickness
    prof_th = 1.
    
    # Camber line stator
    stat = np.array([[UPlot + x[1], 0.], [0, -1], [-x[0], -2.]])
    bez_stator = BezierMatrix.dot(stat)     
    
    # Stator profile pressure side (PS)
    statorPS = np.array([[UPlot + x[1], 0.], [-prof_th/2, -1.], [-x[0], -2.]])
    bez_statorPS = BezierMatrix.dot(statorPS)
    
    # Stator profile suction side (SS)
    statorSS = np.array([[UPlot + x[1], 0.], [prof_th/2., -1.], [-x[0], -2.]])
    bez_statorSS = BezierMatrix.dot(statorSS)
    
    stator = np.vstack((bez_statorSS, bez_statorPS[::-1]))
    
    # Camber line rotor
    rot = np.array([[0., -3.], [-x[0] + UPlot , -4.], [-x[0] + UPlot - x[1], -5.]])
    bez_stator = BezierMatrix.dot(rot)     
    
    # Rotor profile pressure side (PS)
    rotorPS = np.array([[0., -3.], [-x[0] + UPlot - prof_th/2., -4.], [-x[0] + UPlot - x[1], -5.]])
    bez_rotorPS = BezierMatrix.dot(rotorPS)
    
    # Rotor profile suction side (SS)
    rotorSS = np.array([[0., -3.], [-x[0] + UPlot + prof_th/2., -4.], [-x[0] + UPlot - x[1], -5.]])
    bez_rotorSS = BezierMatrix.dot(rotorSS)
    
    rotor = np.vstack((bez_rotorSS, bez_rotorPS[::-1]))
    
    return rotor, stator, x, UPlot # Compressor has rotor first

In [ ]:
# Define plot parameters
scale_fig1 = 1.5  
fontsize = 16

# Define initial parameters
init_pitch_stat = 3
init_pitch_rot = 3

def turbine_gui(r, pitch_stat, pitch_rot, show_velocity=True): 
    
    
    offset_y = 0 if show_velocity == True else 0.25

    # Initialize figure 
    fig1, ax1 = plt.subplots(figsize=(scale_fig1*6.4, scale_fig1*4.8))

    # Set parameters left subplot
    ax1.set_xlabel('$x$', fontsize=fontsize)
    ax1.set_ylabel('$y$', fontsize=fontsize)
    ax1.set_xlim(-10, 10)
    ax1.set_ylim(-7, 2)
    
    [rotor, stator,  x, U] = compressor_blades(r)
    
    for i in range(-10, 11):

        # Draw rotor (blue) at rotor pitch positions and on the top (+0.25)
        ax1.plot(rotor[:,0] + i*pitch_rot, rotor[:,1] + 0.25 - offset_y, lw=3, c='r')
        # Draw stator (red) at stator pitch positions and on the bottom (-0.25)
        ax1.plot(stator[:,0] + i*pitch_stat, stator[:,1] - 0.25 + offset_y, lw=3, c='b')

    if show_velocity:
        offset = 0
        head_length = head_width = 0.3

        # --- Rotor inlet (station 1) ---
        ax1.arrow(offset, 1.5, -x[1]-U, -1,
                color='r', head_length=head_length,
                head_width=head_width, length_includes_head=True,
                label='absolute velocity')
        ax1.arrow(offset, 1.5, -x[1], -1,
                color='b', head_length=head_length,
                head_width=head_width, length_includes_head=True,
                label='relative velocity')
        ax1.arrow(offset - x[1], 0.5, -U, 0,
                color='g', head_length=head_length,
                head_width=head_width, length_includes_head=True,
                label='rotational velocity')

        ax1.arrow(offset, -2, -x[0], -1,  # C2
                color='r', head_length=head_length,
                head_width=head_width, length_includes_head=True)
        ax1.arrow(offset, -2, -x[0] + U, -1,  # W2
                color='b', head_length=head_length,
                head_width=head_width, length_includes_head=True)
        ax1.arrow(offset - x[0] + U, -3, -U, 0,  # U
                color='g', head_length=head_length,
                head_width=head_width, length_includes_head=True)

        ax1.arrow(offset, -5.5, -x[1]-U, -1,  # C3
                color='r', head_length=head_length,
                head_width=head_width, length_includes_head=True)

        ax1.legend()

        
    # Update plot
    
    plt.show()

# Define interactive widgets
radius_slider = widgets.FloatSlider(value=r_midspan, min=r_hub, max=r_tip, step=0.01, description=r'radius R (m)')
pitch_rot_slider = widgets.FloatSlider(value=init_pitch_rot, min=0, max=10, step=0.01, description='Rotor Pitch')
pitch_stat_slider = widgets.FloatSlider(value=init_pitch_stat, min=0, max=10, step=0.01, description='Stator Pitch')
show_velocity_chkbx = widgets.Checkbox(True,  description='Show velocity triangles')
    
# Layout of widgets
topRow = widgets.HBox([radius_slider, show_velocity_chkbx])
bottomRow = widgets.HBox([pitch_rot_slider, pitch_stat_slider])

# Create HTML widget (will be updated dynamically)
textBox = widgets.HTMLMath(value="")

# Callback to update the HTML box when sliders change
def update_text(r, pitch_stat, pitch_rot, show_velocity):
    # Calculate local parameters at radius r
    b1, a2, b2 = calculateBladeAngles(r)
    U_local = U(r)
    phi_local = c_ax / U_local
    psi_local = delta_h0 / (U_local**2)
    
    # Degree of reaction at radius r
    degree_local = 0.5 - 0.5 * phi_local * np.tan(b2)
    
    # Absolute Mach number: M_abs = c / a, where c = sqrt(c_ax^2 + c_theta^2)
    c_theta_local = A2 / r
    c_abs = np.sqrt(c_ax**2 + c_theta_local**2)
    a_sound = np.sqrt(kappa * R_gas * T)  # speed of sound at inlet static temperature
    M_abs = c_abs / a_sound
    
    # Relative Mach number: M_rel = w / a, where w = sqrt(w_ax^2 + w_theta^2)
    # w_ax = c_ax (constant), w_theta = c_theta - U
    w_theta = c_theta_local - U_local
    w_rel = np.sqrt(c_ax**2 + w_theta**2)
    M_rel = w_rel / a_sound
    
    # Format as HTML
    text = (
        f"Calculated parameters at r = {r:.3f} m:<br>"
        f"<ul>"
        f"<li>Blade speed U: {U_local:.2f} m/s</li>"
        f"<li>Axial velocity c<sub>x</sub>: {c_ax:.2f} m/s</li>"
        f"<li>Velocity angles: β₁ = {np.degrees(b1):.2f}°, α₂ = {np.degrees(a2):.2f}°, β₂ = {np.degrees(b2):.2f}°</li>"
        f"<li>Flow coefficient φ: {phi(r_midspan):.3f} [-]</li>"
        f"<li>Loading factor ψ (local): {psi(r_midspan):.3f} [-]</li>"
        f"<li>Degree of reaction R: {degreeOfReaction(r_midspan):.3f} [-]</li>"
        f"<li>Absolute Mach number M<sub>abs</sub>: {M_abs:.3f} [-]</li>"
        f"<li>Relative Mach number M<sub>rel</sub>: {M_rel:.3f} [-]</li>"
        f"</ul>"
    )
    textBox.value = text

# textBox = widgets.HTMLMath(
#     value=f"Calculated parameters: </br><li> Midspan Velocity U: {U(r_midspan):.2f} ms^-1 </li><li> Mid span raiudus: {r_midspan:.2f} m </li><li> Axial velocity: {c_ax:.2f} ms^-1 </li><li> Velocity angles: β₁: {np.degrees(calculateBladeAngles(radius_slider.value)[0]):.2f} °, α₂: {np.degrees(calculateBladeAngles(radius_slider.value)[1]):.2f} °, β₂: {np.degrees(calculateBladeAngles(radius_slider.value)[2]):.2f} ° </li><li> Flow coefficient φ: {phi(r_midspan):.3f} [-]</li><li> Loading factor ψ: {psi(r_midspan):.3f} [-]</li><li> Degree of reaction: {degreeOfReaction:.3f} [-]</li>"
#     )



ui = widgets.VBox([topRow, bottomRow, textBox])


# Activate interactivity with plot!
out = widgets.interactive_output(turbine_gui, 
                                 {'r': radius_slider, 
                                  'pitch_stat': pitch_stat_slider, 
                                  'pitch_rot': pitch_rot_slider, 
                                  'show_velocity': show_velocity_chkbx})
out_text = widgets.interactive_output(update_text, 
                                      {'r': radius_slider, 
                                       'pitch_stat': pitch_stat_slider, 
                                       'pitch_rot': pitch_rot_slider, 
                                       'show_velocity': show_velocity_chkbx})

display(ui, out, out_text)